In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.nhl_scraper import time_remaining, nhl_scraper, shot_scraper2, get_processed_data, get_skater_stats, add_skaters_on_ice, get_corsi_df, get_toi_df, get_attempts_df, tally_xG, tally_corsi, fill_shot_attempts

In [46]:
game_id = 2024021036
shots_df,blocks_df,misses_df,goals_df,hits_df,give_take_df,faceoffs_df,pens_df,shifts_df,players_df,games_df,teams_df = nhl_scraper([game_id])

In [47]:
import pandas as pd

In [48]:
from src.config import MODELS_DIR
import joblib

model = joblib.load(MODELS_DIR / "xgb_v1.pkl")

/home/nolan/anaconda3/envs/reports/lib/python3.13/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/nolan/anaconda3/envs/reports/lib/python3.13/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/nolan/anaconda3/envs/reports/lib/python3.13/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6

In [49]:
temp_df, time_df = shot_scraper2([game_id])
temp_df = get_skater_stats(temp_df)
processed_df = get_processed_data(temp_df)

processed_df

,home,last_play,rebound,rush,home_skaters,away_skaters,x_coord,y_coord,shot_type,zone,position,shooter_hand,shooting_pct,glove_hand,save_pct,angle,shot_on_glove,situation,target
0,Home,hit,No rebound,No rush,5,5,44,39,wrist,O,D,R,0.0464,L,0.898198,40.91,RL,EV,0
1,Home,faceoff,No rebound,No rush,5,5,78,13,snap,O,R,L,0.1260,L,0.898198,49.76,LL,EV,0
2,Home,hit,No rebound,No rush,5,5,71,9,wrist,O,L,R,0.1319,L,0.898198,26.57,RL,EV,0
3,Away,shot-on-goal,No rebound,No rush,5,5,69,19,wrist,O,C,L,0.1420,L,0.905881,43.53,LL,EV,0
4,Home,takeaway,No rebound,No rush,5,5,32,11,slap,O,D,L,0.0446,L,0.898198,10.92,LL,EV,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Home,missed-shot,No rebound,No rush,4,4,80,-1,tip-in,O,L,R,0.1456,L,0.898198,-6.34,RL,EV,0
81,Home,faceoff,No rebound,No rush,5,4,51,4,snap,O,D,R,0.0464,L,0.898198,6.01,RL,PP,1
82,Away,faceoff,No rebound,No rush,4,5,41,-13,slap,O,C,L,0.1111,L,0.905881,-15.15,LL,SH,0
83,Away,giveaway,No rebound,Rush,5,5,32,-1,snap,O,D,R,0.0372,L,0.905881,-1.01,RL,EV,0


In [50]:
full_shots_df = add_skaters_on_ice(processed_df, time_df, shifts_df)
preds = model.predict_proba(full_shots_df.drop(['period', 'time', 'time_seconds', 'game_seconds',
       'home_players', 'away_players'], axis=1))
xG = preds[:,1]
full_shots_df["xG"] = xG
full_shots_df

,home,last_play,rebound,rush,home_skaters,away_skaters,x_coord,y_coord,shot_type,zone,...,shot_on_glove,situation,target,period,time,time_seconds,game_seconds,home_players,away_players,xG
0,Home,hit,No rebound,No rush,5,5,44,39,wrist,O,...,RL,EV,0,1,00:43,43,43,"[8475279, 8475852, 8477353, 8477946, 8481542, ...","[8478043, 8479359, 8480045, 8480839, 8481522, ...",0.008904
1,Home,faceoff,No rebound,No rush,5,5,78,13,snap,O,...,LL,EV,0,1,01:39,99,99,"[8475225, 8475718, 8475765, 8475852, 8476979, ...","[8475722, 8477949, 8480045, 8480196, 8480802, ...",0.204576
2,Home,hit,No rebound,No rush,5,5,71,9,wrist,O,...,RL,EV,0,1,01:56,116,116,"[8474141, 8475718, 8475765, 8475852, 8476979, ...","[8475722, 8477949, 8480045, 8480196, 8480802, ...",0.146474
3,Away,shot-on-goal,No rebound,No rush,5,5,69,19,wrist,O,...,LL,EV,0,1,02:06,126,126,"[8474141, 8475718, 8475765, 8475852, 8482762, ...","[8477365, 8478413, 8480045, 8480802, 8481524, ...",0.052182
4,Home,takeaway,No rebound,No rush,5,5,32,11,slap,O,...,LL,EV,0,1,02:57,177,177,"[8474141, 8475279, 8475852, 8479337, 8481542, ...","[8477365, 8478413, 8480045, 8482097, 8482671, ...",0.029260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Home,missed-shot,No rebound,No rush,4,4,80,-1,tip-in,O,...,RL,EV,0,3,13:26,806,3206,"[8475852, 8477946, 8481607, 8482078, 8482762]","[8480045, 8480802, 8480839, 8481524, 8483468]",0.130279
81,Home,faceoff,No rebound,No rush,5,4,51,4,snap,O,...,RL,PP,1,3,14:31,871,3271,"[8474141, 8475225, 8475718, 8475852, 8476979, ...","[8475722, 8477365, 8478043, 8479359, 8480045, ...",0.077886
82,Away,faceoff,No rebound,No rush,4,5,41,-13,slap,O,...,LL,SH,0,3,18:32,1112,3512,"[8475279, 8475852, 8477353, 8481542, 8483464]","[8479359, 8480045, 8480802, 8480839, 8481522, ...",0.028868
83,Away,giveaway,No rebound,Rush,5,5,32,-1,snap,O,...,RL,EV,0,3,19:15,1155,3555,"[8474141, 8475279, 8475852, 8481542, 8481725, ...","[8477365, 8478043, 8479359, 8480045, 8481522, ...",0.015626


In [51]:
xG_totals = tally_xG(full_shots_df, players_df)
xG_totals["xGF%"] = xG_totals["xG_for"] / (xG_totals["xG_for"] + xG_totals["xG_against"])
xG_totals

,player_id,period,xG_for,xG_against,name,team,position,xGF%
0,8481607,1,0.593940,0.037513,Albert Johansson,17,D,0.940593
1,8481607,2,0.210643,0.109572,Albert Johansson,17,D,0.657816
2,8481607,3,0.922114,0.164356,Albert Johansson,17,D,0.848725
3,8479337,1,0.729466,0.146118,Alex DeBrincat,17,R,0.833120
4,8479337,2,0.834753,0.354618,Alex DeBrincat,17,R,0.701844
...,...,...,...,...,...,...,...,...
109,8475765,2,0.451430,0.328987,Vladimir Tarasenko,17,R,0.578447
110,8475765,3,0.502552,0.163988,Vladimir Tarasenko,17,R,0.753971
111,8484145,1,0.409235,0.656757,Zach Benson,7,L,0.383901
112,8484145,2,0.196494,0.174875,Zach Benson,7,L,0.529108


In [52]:
attempts_df = get_attempts_df(shots_df, misses_df, blocks_df, goals_df)
full_attempts_df = fill_shot_attempts(attempts_df, players_df)
corsi_df = get_corsi_df(attempts_df=attempts_df, shifts_df=shifts_df)
corsi_totals = tally_corsi(corsi_df, players_df, game_id)

In [53]:
corsi_totals["CF%"] = corsi_totals["corsi_for"] / (corsi_totals["corsi_for"] + corsi_totals["corsi_against"])

In [54]:
corsi_totals

,player_id,period,corsi_for,corsi_against,name,team,position,CF%
0,8481607,1,8,3,Albert Johansson,17,D,0.727273
1,8481607,2,5,5,Albert Johansson,17,D,0.500000
2,8481607,3,7,6,Albert Johansson,17,D,0.538462
3,8479337,1,9,4,Alex DeBrincat,17,R,0.692308
4,8479337,2,9,7,Alex DeBrincat,17,R,0.562500
...,...,...,...,...,...,...,...,...
109,8475765,2,7,2,Vladimir Tarasenko,17,R,0.777778
110,8475765,3,3,3,Vladimir Tarasenko,17,R,0.500000
111,8484145,1,6,8,Zach Benson,7,L,0.428571
112,8484145,2,8,5,Zach Benson,7,L,0.615385


In [55]:
corsi_totals["name"].unique()

array(['Albert Johansson', 'Alex DeBrincat', 'Alex Tuch',
       'Beck Malenstyn', 'Ben Chiarot', 'Bowen Byram', 'Connor Clifton',
       'Craig Smith', 'Dominik Shine', 'Dylan Larkin', 'Elmer Soderblom',
       'Erik Gustafsson', 'J.T. Compher', 'Jack Quinn', 'Jacob Bryson',
       'Jason Zucker', 'Jiri Kulich', 'Jordan Greenway', 'Josh Norris',
       'Justin Holl', 'Lucas Raymond', 'Marco Kasper',
       'Mattias Samuelsson', 'Michael Rasmussen', 'Moritz Seider',
       'Owen Power', 'Patrick Kane', 'Petr Mrazek', 'Peyton Krebs',
       'Rasmus Dahlin', 'Ryan McLeod', 'Sam Lafferty', 'Simon Edvinsson',
       'Tage Thompson', 'Tyler Motte', 'Ukko-Pekka Luukkonen',
       'Vladimir Tarasenko', 'Zach Benson'], dtype=object)

In [56]:
full_attempts_df["name"].unique()

array(['Alex DeBrincat', 'Alex Tuch', 'Ben Chiarot', 'Bowen Byram',
       'Connor Clifton', 'Craig Smith', 'Dominik Shine', 'Dylan Larkin',
       'Elmer Soderblom', 'Erik Gustafsson', 'J.T. Compher', 'Jack Quinn',
       'Jacob Bryson', 'Jason Zucker', 'Jiri Kulich', 'Jordan Greenway',
       'Josh Norris', 'Justin Holl', 'Lucas Raymond', 'Marco Kasper',
       'Mattias Samuelsson', 'Michael Rasmussen', 'Moritz Seider',
       'Owen Power', 'Patrick Kane', 'Peyton Krebs', 'Rasmus Dahlin',
       'Ryan McLeod', 'Simon Edvinsson', 'Tage Thompson', 'Tyler Motte',
       'Vladimir Tarasenko', 'Zach Benson'], dtype=object)

In [57]:
xG_corsi_df = pd.merge(xG_totals, corsi_totals, how="outer", on=["name", "period"])

In [58]:
full_attempts_df

,player_id,period,total_attempts,name,team,position
0,8479337,1,1,Alex DeBrincat,17,R
1,8479337,2,2,Alex DeBrincat,17,R
2,8479337,3,1,Alex DeBrincat,17,R
3,8477949,1,3,Alex Tuch,7,R
4,8477949,2,0,Alex Tuch,7,R
...,...,...,...,...,...,...
94,8475765,2,3,Vladimir Tarasenko,17,R
95,8475765,3,0,Vladimir Tarasenko,17,R
96,8484145,1,0,Zach Benson,7,L
97,8484145,2,1,Zach Benson,7,L


In [59]:
xG_corsi_df

,player_id_x,period,xG_for,xG_against,name,team_x,position_x,xGF%,player_id_y,corsi_for,corsi_against,team_y,position_y,CF%
0,8481607,1,0.593940,0.037513,Albert Johansson,17,D,0.940593,8481607,8,3,17,D,0.727273
1,8481607,2,0.210643,0.109572,Albert Johansson,17,D,0.657816,8481607,5,5,17,D,0.500000
2,8481607,3,0.922114,0.164356,Albert Johansson,17,D,0.848725,8481607,7,6,17,D,0.538462
3,8479337,1,0.729466,0.146118,Alex DeBrincat,17,R,0.833120,8479337,9,4,17,R,0.692308
4,8479337,2,0.834753,0.354618,Alex DeBrincat,17,R,0.701844,8479337,9,7,17,R,0.562500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,8475765,2,0.451430,0.328987,Vladimir Tarasenko,17,R,0.578447,8475765,7,2,17,R,0.777778
110,8475765,3,0.502552,0.163988,Vladimir Tarasenko,17,R,0.753971,8475765,3,3,17,R,0.500000
111,8484145,1,0.409235,0.656757,Zach Benson,7,L,0.383901,8484145,6,8,7,L,0.428571
112,8484145,2,0.196494,0.174875,Zach Benson,7,L,0.529108,8484145,8,5,7,L,0.615385


In [83]:
final_df = pd.merge(
    xG_corsi_df,
    full_attempts_df,
    left_on=["player_id_x", "period"],
    right_on=["player_id", "period"],
    how="left"
)

final_df["total_attempts"] = final_df["total_attempts"].fillna(0).astype(int)

In [61]:
final_df.columns

Index(['player_id_x', 'period', 'xG_for', 'xG_against', 'name_x', 'team_x',
       'position_x', 'xGF%', 'player_id_y', 'corsi_for', 'corsi_against',
       'team_y', 'position_y', 'CF%', 'player_id', 'total_attempts', 'name_y',
       'team', 'position'],
      dtype='object')

In [84]:
final_df = final_df.drop(["player_id", "position_x", "position_y", "player_id_y", "name_y", "position", "team", "team_y"], axis=1)

In [88]:
final_df.rename(columns={"player_id_x": "player_id", "name_x": "name", "team_x": "team"}, inplace=True)

In [ ]:
final_df = pd.merge(final_df, teams_df, left_on="team", right_on="team_id").drop("team", axis=1)

In [89]:
final_df

,player_id,period,xG_for,xG_against,name,xGF%,corsi_for,corsi_against,CF%,total_attempts,team_id,team_abbrev,city,name_y,conference,division
0,8481607,1,0.593940,0.037513,Albert Johansson,0.940593,8,3,0.727273,0,17,DET,Detroit,Red Wings,Eastern,Atlantic
1,8481607,2,0.210643,0.109572,Albert Johansson,0.657816,5,5,0.500000,0,17,DET,Detroit,Red Wings,Eastern,Atlantic
2,8481607,3,0.922114,0.164356,Albert Johansson,0.848725,7,6,0.538462,0,17,DET,Detroit,Red Wings,Eastern,Atlantic
3,8479337,1,0.729466,0.146118,Alex DeBrincat,0.833120,9,4,0.692308,1,17,DET,Detroit,Red Wings,Eastern,Atlantic
4,8479337,2,0.834753,0.354618,Alex DeBrincat,0.701844,9,7,0.562500,2,17,DET,Detroit,Red Wings,Eastern,Atlantic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,8475765,2,0.451430,0.328987,Vladimir Tarasenko,0.578447,7,2,0.777778,3,17,DET,Detroit,Red Wings,Eastern,Atlantic
110,8475765,3,0.502552,0.163988,Vladimir Tarasenko,0.753971,3,3,0.500000,0,17,DET,Detroit,Red Wings,Eastern,Atlantic
111,8484145,1,0.409235,0.656757,Zach Benson,0.383901,6,8,0.428571,0,7,BUF,Buffalo,Sabres,Eastern,Atlantic
112,8484145,2,0.196494,0.174875,Zach Benson,0.529108,8,5,0.615385,1,7,BUF,Buffalo,Sabres,Eastern,Atlantic


In [92]:
final_df.columns

Index(['player_id', 'period', 'xG_for', 'xG_against', 'name', 'xGF%',
       'corsi_for', 'corsi_against', 'CF%', 'total_attempts', 'team_id',
       'team_abbrev', 'city', 'name_y', 'conference', 'division'],
      dtype='object')

In [91]:
final_df["corsi_against"].dtype

dtype('int64')

In [90]:
from src.config import DATA_DIR

final_df.to_csv(os.path.join(DATA_DIR, "shot_info.csv"), index=False)

In [77]:
games_df

,game,away_id,away_score,home_id,home_score,winner,loser
0,2024021036,7,3,17,7,17,7


In [78]:
teams_df

,team_id,team_abbrev,city,name,conference,division
0,17,DET,Detroit,Red Wings,Eastern,Atlantic
1,7,BUF,Buffalo,Sabres,Eastern,Atlantic


In [93]:
client = NHLClient()

In [94]:
client.teams.teams_info()

[{'conference': {'abbr': 'W', 'name': 'Western'},
  'division': {'abbr': 'C', 'name': 'Central'},
  'name': 'Winnipeg Jets',
  'common_name': 'Jets',
  'abbr': 'WPG',
  'logo': 'https://assets.nhle.com/logos/nhl/svg/WPG_light.svg',
  'franchise_id': 35},
 {'conference': {'abbr': 'E', 'name': 'Eastern'},
  'division': {'abbr': 'M', 'name': 'Metropolitan'},
  'name': 'Washington Capitals',
  'common_name': 'Capitals',
  'abbr': 'WSH',
  'logo': 'https://assets.nhle.com/logos/nhl/svg/WSH_secondary_light.svg',
  'franchise_id': 24},
 {'conference': {'abbr': 'W', 'name': 'Western'},
  'division': {'abbr': 'P', 'name': 'Pacific'},
  'name': 'Vegas Golden Knights',
  'common_name': 'Golden Knights',
  'abbr': 'VGK',
  'logo': 'https://assets.nhle.com/logos/nhl/svg/VGK_light.svg',
  'franchise_id': 38},
 {'conference': {'abbr': 'E', 'name': 'Eastern'},
  'division': {'abbr': 'A', 'name': 'Atlantic'},
  'name': 'Toronto Maple Leafs',
  'common_name': 'Maple Leafs',
  'abbr': 'TOR',
  'logo': 'h

In [96]:
toi_df = get_toi_df(shifts_df, players_df)

In [97]:
full_shots_df

,home,last_play,rebound,rush,home_skaters,away_skaters,x_coord,y_coord,shot_type,zone,...,shot_on_glove,situation,target,period,time,time_seconds,game_seconds,home_players,away_players,xG
0,Home,hit,No rebound,No rush,5,5,44,39,wrist,O,...,RL,EV,0,1,00:43,43,43,"[8475279, 8475852, 8477353, 8477946, 8481542, ...","[8478043, 8479359, 8480045, 8480839, 8481522, ...",0.008904
1,Home,faceoff,No rebound,No rush,5,5,78,13,snap,O,...,LL,EV,0,1,01:39,99,99,"[8475225, 8475718, 8475765, 8475852, 8476979, ...","[8475722, 8477949, 8480045, 8480196, 8480802, ...",0.204576
2,Home,hit,No rebound,No rush,5,5,71,9,wrist,O,...,RL,EV,0,1,01:56,116,116,"[8474141, 8475718, 8475765, 8475852, 8476979, ...","[8475722, 8477949, 8480045, 8480196, 8480802, ...",0.146474
3,Away,shot-on-goal,No rebound,No rush,5,5,69,19,wrist,O,...,LL,EV,0,1,02:06,126,126,"[8474141, 8475718, 8475765, 8475852, 8482762, ...","[8477365, 8478413, 8480045, 8480802, 8481524, ...",0.052182
4,Home,takeaway,No rebound,No rush,5,5,32,11,slap,O,...,LL,EV,0,1,02:57,177,177,"[8474141, 8475279, 8475852, 8479337, 8481542, ...","[8477365, 8478413, 8480045, 8482097, 8482671, ...",0.029260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Home,missed-shot,No rebound,No rush,4,4,80,-1,tip-in,O,...,RL,EV,0,3,13:26,806,3206,"[8475852, 8477946, 8481607, 8482078, 8482762]","[8480045, 8480802, 8480839, 8481524, 8483468]",0.130279
81,Home,faceoff,No rebound,No rush,5,4,51,4,snap,O,...,RL,PP,1,3,14:31,871,3271,"[8474141, 8475225, 8475718, 8475852, 8476979, ...","[8475722, 8477365, 8478043, 8479359, 8480045, ...",0.077886
82,Away,faceoff,No rebound,No rush,4,5,41,-13,slap,O,...,LL,SH,0,3,18:32,1112,3512,"[8475279, 8475852, 8477353, 8481542, 8483464]","[8479359, 8480045, 8480802, 8480839, 8481522, ...",0.028868
83,Away,giveaway,No rebound,Rush,5,5,32,-1,snap,O,...,RL,EV,0,3,19:15,1155,3555,"[8474141, 8475279, 8475852, 8481542, 8481725, ...","[8477365, 8478043, 8479359, 8480045, 8481522, ...",0.015626


In [101]:
box_score = client.game_center.boxscore(game_id)

In [102]:
box_score

{'id': 2024021036,
 'season': 20242025,
 'gameType': 2,
 'limitedScoring': False,
 'gameDate': '2025-03-12',
 'venue': {'default': 'Little Caesars Arena'},
 'venueLocation': {'default': 'Detroit'},
 'startTimeUTC': '2025-03-12T23:30:00Z',
 'easternUTCOffset': '-04:00',
 'venueUTCOffset': '-04:00',
 'tvBroadcasts': [{'id': 385,
   'market': 'N',
   'countryCode': 'US',
   'network': 'TNT',
   'sequenceNumber': 11},
  {'id': 501,
   'market': 'N',
   'countryCode': 'US',
   'network': 'truTV',
   'sequenceNumber': 14},
  {'id': 519,
   'market': 'N',
   'countryCode': 'US',
   'network': 'MAX',
   'sequenceNumber': 18},
  {'id': 281,
   'market': 'N',
   'countryCode': 'CA',
   'network': 'TVAS',
   'sequenceNumber': 109},
  {'id': 562,
   'market': 'H',
   'countryCode': 'US',
   'network': 'FDSNDET',
   'sequenceNumber': 271}],
 'gameState': 'OFF',
 'gameScheduleState': 'OK',
 'periodDescriptor': {'number': 3,
  'periodType': 'REG',
  'maxRegulationPeriods': 3},
 'regPeriods': 3,
 'awa

In [104]:
away_team_info = box_score["awayTeam"]
away_team = away_team_info["abbrev"]
away_score = away_team_info["score"]
away_sog = away_team_info["sog"]

home_team_info = box_score["homeTeam"]
home_team = home_team_info["abbrev"]
home_score = home_team_info["score"]
home_sog = home_team_info["sog"]

data = {
        "team": [away_team, home_team],
        "score": [away_score, home_score],
        "sog": [away_sog, home_sog]
        }

score_df = pd.DataFrame(data)

In [106]:
player_stats = box_score["playerByGameStats"]
player_stats.keys()

dict_keys(['awayTeam', 'homeTeam'])

In [109]:
away_forward_stats = player_stats["awayTeam"]["forwards"]
away_defense_stats = player_stats["awayTeam"]["defense"]
away_goalie_stats = player_stats["awayTeam"]["goalies"]

skater_rows = []
goalie_rows = []

for player in away_forward_stats:
    player_id = player["playerId"]
    goals = player["goals"]
    assists = player["assists"]
    points = player["points"]
    plus_minus = player["plusMinus"]
    pim = player["pim"]
    hits = player["hits"]
    pp_goals = player["powerPlayGoals"]
    sog = player["sog"]
    fo_pct = player["faceoffWinningPctg"]
    blocked_shots = player["blockedShots"]
    giveaways = player["giveaways"]
    takeaways = player["takeaways"]
    skater_rows.append((player_id, goals, assists, points, plus_minus, pim, hits, pp_goals, sog, fo_pct, blocked_shots, giveaways, takeaways))

for player in away_defense_stats:
    player_id = player["playerId"]
    goals = player["goals"]
    assists = player["assists"]
    points = player["points"]
    plus_minus = player["plusMinus"]
    pim = player["pim"]
    hits = player["hits"]
    pp_goals = player["powerPlayGoals"]
    sog = player["sog"]
    fo_pct = player["faceoffWinningPctg"]
    blocked_shots = player["blockedShots"]
    giveaways = player["giveaways"]
    takeaways = player["takeaways"]
    skater_rows.append((player_id, goals, assists, points, plus_minus, pim, hits, pp_goals, sog, fo_pct, blocked_shots, giveaways, takeaways))

for player in away_goalie_stats:
    player_id = player["playerId"]
    ga = player["goalsAgainst"]
    sa = player["shotsAgainst"]
    saves = player["saves"]
    goalie_rows.append((player_id, ga, sa, saves))

skater_h = ["player_id", "goals", "assists", "points", "plus_minus", "pim", "hits", "pp_goals", "sog", "fo_pct", "blocked_shots", "giveaways", "takeaways"]
goalie_h = ["player_id", "goals_against", "shots_against", "saves"]

skater_box_score = pd.DataFrame(skater_rows, columns=skater_h)
goalie_box_score = pd.DataFrame(goalie_rows, columns=goalie_h)

In [110]:
skater_box_score

,player_id,goals,assists,points,plus_minus,pim,hits,pp_goals,sog,fo_pct,blocked_shots,giveaways,takeaways
0,8484145,1,1,2,2,12,0,1,2,0.000000,0,0,0
1,8478413,0,0,0,-2,14,5,0,1,1.000000,0,1,0
2,8480064,1,0,1,-1,10,1,0,1,0.625000,1,0,0
3,8475722,0,0,0,-2,2,1,0,1,0.000000,0,1,0
4,8481522,0,0,0,1,0,2,0,0,0.666667,1,0,0
5,8483468,0,0,0,-1,2,0,0,2,0.000000,1,0,0
6,8482097,0,0,0,-2,0,0,0,1,0.000000,0,1,0
7,8479359,0,0,0,-1,2,3,0,0,0.000000,1,0,0
8,8480802,0,1,1,-1,2,0,0,2,0.529412,2,2,0
9,8479420,1,1,2,2,10,0,0,3,0.500000,0,1,0


In [111]:
goalie_box_score

,player_id,goals_against,shots_against,saves
0,8480045,7,29,22
1,8473503,0,0,0


In [115]:
pd.merge(pd.merge(skater_box_score, players_df, how="left", on="player_id"), teams_df, left_on="team", right_on="team_id", how="left")

,player_id,goals,assists,points,plus_minus,pim,hits,pp_goals,sog,fo_pct,...,takeaways,name_x,team,position,team_id,team_abbrev,city,name_y,conference,division
0,8484145,1,1,2,2,12,0,1,2,0.000000,...,0,Zach Benson,7,L,7,BUF,Buffalo,Sabres,Eastern,Atlantic
1,8478413,0,0,0,-2,14,5,0,1,1.000000,...,0,Jordan Greenway,7,L,7,BUF,Buffalo,Sabres,Eastern,Atlantic
2,8480064,1,0,1,-1,10,1,0,1,0.625000,...,0,Josh Norris,7,C,7,BUF,Buffalo,Sabres,Eastern,Atlantic
3,8475722,0,0,0,-2,2,1,0,1,0.000000,...,0,Jason Zucker,7,L,7,BUF,Buffalo,Sabres,Eastern,Atlantic
4,8481522,0,0,0,1,0,2,0,0,0.666667,...,0,Peyton Krebs,7,C,7,BUF,Buffalo,Sabres,Eastern,Atlantic
5,8483468,0,0,0,-1,2,0,0,2,0.000000,...,0,Jiri Kulich,7,C,7,BUF,Buffalo,Sabres,Eastern,Atlantic
6,8482097,0,0,0,-2,0,0,0,1,0.000000,...,0,Jack Quinn,7,R,7,BUF,Buffalo,Sabres,Eastern,Atlantic
7,8479359,0,0,0,-1,2,3,0,0,0.000000,...,0,Beck Malenstyn,7,L,7,BUF,Buffalo,Sabres,Eastern,Atlantic
8,8480802,0,1,1,-1,2,0,0,2,0.529412,...,0,Ryan McLeod,7,C,7,BUF,Buffalo,Sabres,Eastern,Atlantic
9,8479420,1,1,2,2,10,0,0,3,0.500000,...,0,Tage Thompson,7,C,7,BUF,Buffalo,Sabres,Eastern,Atlantic


In [114]:
teams_df

,team_id,team_abbrev,city,name,conference,division
0,17,DET,Detroit,Red Wings,Eastern,Atlantic
1,7,BUF,Buffalo,Sabres,Eastern,Atlantic


In [122]:
from src.nhl_scraper import get_box_score_dfs

In [123]:
score_df, skater_box_score, goalie_box_score = get_box_score_dfs(game_id=game_id, players_df=players_df, teams_df=teams_df)

In [124]:
score_df

,team,score,sog
0,BUF,3,26
1,DET,7,29


In [125]:
skater_box_score

,player_id,goals,assists,points,plus_minus,pim,hits,pp_goals,sog,fo_pct,...,takeaways,name_x,team,position,team_id,team_abbrev,city,name_y,conference,division
0,8484145,1,1,2,2,12,0,1,2,0.000000,...,0,Zach Benson,7,L,7,BUF,Buffalo,Sabres,Eastern,Atlantic
1,8478413,0,0,0,-2,14,5,0,1,1.000000,...,0,Jordan Greenway,7,L,7,BUF,Buffalo,Sabres,Eastern,Atlantic
2,8480064,1,0,1,-1,10,1,0,1,0.625000,...,0,Josh Norris,7,C,7,BUF,Buffalo,Sabres,Eastern,Atlantic
3,8475722,0,0,0,-2,2,1,0,1,0.000000,...,0,Jason Zucker,7,L,7,BUF,Buffalo,Sabres,Eastern,Atlantic
4,8481522,0,0,0,1,0,2,0,0,0.666667,...,0,Peyton Krebs,7,C,7,BUF,Buffalo,Sabres,Eastern,Atlantic
5,8483468,0,0,0,-1,2,0,0,2,0.000000,...,0,Jiri Kulich,7,C,7,BUF,Buffalo,Sabres,Eastern,Atlantic
6,8482097,0,0,0,-2,0,0,0,1,0.000000,...,0,Jack Quinn,7,R,7,BUF,Buffalo,Sabres,Eastern,Atlantic
7,8479359,0,0,0,-1,2,3,0,0,0.000000,...,0,Beck Malenstyn,7,L,7,BUF,Buffalo,Sabres,Eastern,Atlantic
8,8480802,0,1,1,-1,2,0,0,2,0.529412,...,0,Ryan McLeod,7,C,7,BUF,Buffalo,Sabres,Eastern,Atlantic
9,8479420,1,1,2,2,10,0,0,3,0.500000,...,0,Tage Thompson,7,C,7,BUF,Buffalo,Sabres,Eastern,Atlantic


In [126]:
goalie_box_score

,player_id,goals_against,shots_against,saves,save_pct,name_x,team,position,team_id,team_abbrev,city,name_y,conference,division
0,8480045,7,29,22,0.759,Ukko-Pekka Luukkonen,7,G,7,BUF,Buffalo,Sabres,Eastern,Atlantic
1,8473503,0,0,0,NaN,James Reimer,7,G,7,BUF,Buffalo,Sabres,Eastern,Atlantic
2,8475660,0,0,0,NaN,Cam Talbot,17,G,17,DET,Detroit,Red Wings,Eastern,Atlantic
3,8475852,3,26,23,0.885,Petr Mrazek,17,G,17,DET,Detroit,Red Wings,Eastern,Atlantic


In [127]:
full_shots_df

,home,last_play,rebound,rush,home_skaters,away_skaters,x_coord,y_coord,shot_type,zone,...,shot_on_glove,situation,target,period,time,time_seconds,game_seconds,home_players,away_players,xG
0,Home,hit,No rebound,No rush,5,5,44,39,wrist,O,...,RL,EV,0,1,00:43,43,43,"[8475279, 8475852, 8477353, 8477946, 8481542, ...","[8478043, 8479359, 8480045, 8480839, 8481522, ...",0.008904
1,Home,faceoff,No rebound,No rush,5,5,78,13,snap,O,...,LL,EV,0,1,01:39,99,99,"[8475225, 8475718, 8475765, 8475852, 8476979, ...","[8475722, 8477949, 8480045, 8480196, 8480802, ...",0.204576
2,Home,hit,No rebound,No rush,5,5,71,9,wrist,O,...,RL,EV,0,1,01:56,116,116,"[8474141, 8475718, 8475765, 8475852, 8476979, ...","[8475722, 8477949, 8480045, 8480196, 8480802, ...",0.146474
3,Away,shot-on-goal,No rebound,No rush,5,5,69,19,wrist,O,...,LL,EV,0,1,02:06,126,126,"[8474141, 8475718, 8475765, 8475852, 8482762, ...","[8477365, 8478413, 8480045, 8480802, 8481524, ...",0.052182
4,Home,takeaway,No rebound,No rush,5,5,32,11,slap,O,...,LL,EV,0,1,02:57,177,177,"[8474141, 8475279, 8475852, 8479337, 8481542, ...","[8477365, 8478413, 8480045, 8482097, 8482671, ...",0.029260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Home,missed-shot,No rebound,No rush,4,4,80,-1,tip-in,O,...,RL,EV,0,3,13:26,806,3206,"[8475852, 8477946, 8481607, 8482078, 8482762]","[8480045, 8480802, 8480839, 8481524, 8483468]",0.130279
81,Home,faceoff,No rebound,No rush,5,4,51,4,snap,O,...,RL,PP,1,3,14:31,871,3271,"[8474141, 8475225, 8475718, 8475852, 8476979, ...","[8475722, 8477365, 8478043, 8479359, 8480045, ...",0.077886
82,Away,faceoff,No rebound,No rush,4,5,41,-13,slap,O,...,LL,SH,0,3,18:32,1112,3512,"[8475279, 8475852, 8477353, 8481542, 8483464]","[8479359, 8480045, 8480802, 8480839, 8481522, ...",0.028868
83,Away,giveaway,No rebound,Rush,5,5,32,-1,snap,O,...,RL,EV,0,3,19:15,1155,3555,"[8474141, 8475279, 8475852, 8481542, 8481725, ...","[8477365, 8478043, 8479359, 8480045, 8481522, ...",0.015626


In [128]:
final_df

,player_id,period,xG_for,xG_against,name,xGF%,corsi_for,corsi_against,CF%,total_attempts,team_id,team_abbrev,city,name_y,conference,division
0,8481607,1,0.593940,0.037513,Albert Johansson,0.940593,8,3,0.727273,0,17,DET,Detroit,Red Wings,Eastern,Atlantic
1,8481607,2,0.210643,0.109572,Albert Johansson,0.657816,5,5,0.500000,0,17,DET,Detroit,Red Wings,Eastern,Atlantic
2,8481607,3,0.922114,0.164356,Albert Johansson,0.848725,7,6,0.538462,0,17,DET,Detroit,Red Wings,Eastern,Atlantic
3,8479337,1,0.729466,0.146118,Alex DeBrincat,0.833120,9,4,0.692308,1,17,DET,Detroit,Red Wings,Eastern,Atlantic
4,8479337,2,0.834753,0.354618,Alex DeBrincat,0.701844,9,7,0.562500,2,17,DET,Detroit,Red Wings,Eastern,Atlantic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,8475765,2,0.451430,0.328987,Vladimir Tarasenko,0.578447,7,2,0.777778,3,17,DET,Detroit,Red Wings,Eastern,Atlantic
110,8475765,3,0.502552,0.163988,Vladimir Tarasenko,0.753971,3,3,0.500000,0,17,DET,Detroit,Red Wings,Eastern,Atlantic
111,8484145,1,0.409235,0.656757,Zach Benson,0.383901,6,8,0.428571,0,7,BUF,Buffalo,Sabres,Eastern,Atlantic
112,8484145,2,0.196494,0.174875,Zach Benson,0.529108,8,5,0.615385,1,7,BUF,Buffalo,Sabres,Eastern,Atlantic


In [130]:
from src.nhl_scraper import get_and_save_data_for_tableau

get_and_save_data_for_tableau(game_id)

/home/nolan/anaconda3/envs/reports/lib/python3.13/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/nolan/anaconda3/envs/reports/lib/python3.13/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/nolan/anaconda3/envs/reports/lib/python3.13/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6

Saved shot_info.csv!
Saved toi_info.csv!
Saved score_info.csv!
Saved skater_box_info.csv!
Saved goalie_box_info.csv!
Saved shot_location_info.csv!


(     player_id  period    xG_for  xG_against              name_x      xGF%  \
 0      8481607       1  0.593940    0.037513    Albert Johansson  0.940593   
 1      8481607       2  0.210643    0.109572    Albert Johansson  0.657816   
 2      8481607       3  0.922114    0.164356    Albert Johansson  0.848725   
 3      8479337       1  0.729466    0.146118      Alex DeBrincat  0.833120   
 4      8479337       2  0.834753    0.354618      Alex DeBrincat  0.701844   
 ..         ...     ...       ...         ...                 ...       ...   
 109    8475765       2  0.451430    0.328987  Vladimir Tarasenko  0.578447   
 110    8475765       3  0.502552    0.163988  Vladimir Tarasenko  0.753971   
 111    8484145       1  0.409235    0.656757         Zach Benson  0.383901   
 112    8484145       2  0.196494    0.174875         Zach Benson  0.529108   
 113    8484145       3  0.103047    0.667508         Zach Benson  0.133731   
 
      corsi_for  corsi_against       CF%  total_at